# Reading SPWLA file

The 3 methods for extracting data are:

- Looping over the file and applying string methods, etc. >>> this is a naive approach, very fiddly.
- Using regex to extract everything at once. >>> I have this 'sort of' working, see below.
- **Using a PEG parser.** >>> this notebook.

In [362]:
with open('../data/core_analysis_example.spwla') as f:
    text = f.read()

In [407]:
test = text[:1479]

print(test)

10     2                                                                                                                       
    9999/9-9                                Norway                                            9Sep99
    Weatherford-Labs
15    10   10
          1507      1602  2031   0Weatherford-Labs    Nitrogen Permeability, Hor.
          1512      1602  2031   0Weatherford-Labs    Klinkenberg corrected gas perm, Hor.
          1510      1602  2031   0Weatherford-Labs    Nitrogen Permeability, Vert.
          1515      1602  2031   0Weatherford-Labs    Klinkenberg corrected gas perm, Vert.
          1402      1211  3084   0Weatherford-Labs    Porosity, Horizontal PLUG
          1403      1211  3084   0Weatherford-Labs    Porosity, Vertical PLUG
          1401      1212  3084   0Weatherford-Labs    Porosity, Summation
          1302      1103  3085   0Weatherford-Labs    CORE Oil Saturation
          1301      1103  3085   0Weatherford-Labs    CORE Water Saturation
      

### Observations

- Some lines are 128 characters wide
- Some of the data is unidentifiable
- This is probably a job for striplog
- The info after the record type (10, 15, 20, 30, etc) seems to be the number of lines (and fields per line, perhaps) in that record, which is redundant information (can just read until the next record type flag)

## Parsing SPWLA file

Let's try using `parsimonious`. There are examples to build from:

- [In the docs](https://github.com/erikrose/parsimonious)
- [On StackOverflow](https://stackoverflow.com/questions/47982949/how-to-parse-complex-text-files-using-python)

First we have to write a **parsing expression grammar**. PEGs are similar to a _context-free grammar_ (i.e. one in which the terms are uniquely defined, and don't depend on context), but there is zero ambiguity. PEGs are good at describing artificial languages (whereas CFGs are better for human languages). [Read more.](https://en.wikipedia.org/wiki/Parsing_expression_grammar)

Let's try writing the PEG for this file format...

In [408]:
from parsimonious import Grammar

grammar = Grammar(
    r"""
    file          = file_block field_block range_block data_blocks+
    
    file_block    = "10" ws number ws file_data ws
    field_block   = "15" ws number ws number ws (field_row ws)+
    range_block   = "20" ws number ws range_data ws
    data_blocks   = depth_block ws (descr_block ws)? data_block ws
    
    file_data     = well_name ws country ws date ws company ws
    field_row     = number ws number ws number ws "0" company ws field_name
    range_data    = number ws number ws start_depth ws stop_depth ws number ws "1"

    depth_block   = "30" ws "1" ws depth ws number ws number ws

    descr_block   = "36" ws "1" ws "1" ws description ws

    data_block    = data_header data_row
    data_header   = "40" ws "1" ws record_count ws
    data_row      = (ws datum)* newline

    start_depth   = number+
    stop_depth    = number+
    record_count  = number+
    datum         = number+
    depth         = number+
    description   = sentence+
    field_name    = sentence+

    ws            = ~r"\s*"
    newline       = ~r"\n*"
    number        = ~r"[-.0-9]+"
    well_name     = ~r"[-/0-9]+"
    country       = ~r"[A-Z]\w+"
    company       = ~r"[-_a-zA-Z]+"
    date          = ~r"[a-zA-Z0-9]+"
    sentence      = ~r"[-., /A-Za-z]+"
    """
)

In [409]:
tree = grammar.parse(test)

In [415]:
from parsimonious import NodeVisitor
import pandas as pd


class FileVisitor(NodeVisitor):

    file = {}
    file['fields'] = []
    file['data'] = []
    file['depth'] = []
    file['description'] = []
    
    def visit_file_data(self, node, visited_children):
        well, _, country, _, date, _, company, _ = node.children
        self.file['well_name'] = well.text
        self.file['country'] = country.text
        self.file['company'] = company.text
        try:
            self.file['date'] = pd.to_datetime(date.text).isoformat()
        except (TypeError, ValueError) as e:
            self.file['date'] = date.text

    def visit_field_row(self, node, visited_children):
        *_, field = node.children
        self.file['fields'].append(field.text.strip())

    def visit_depth_block(self, node, visited_children):
        _, _, _, _, depth, *_ = node.children
        self.file['depth'].append(float(depth.text))

    def visit_data_row(self, node, visited_children):
        data = []
        for child in node.children:
            print(child.expr.name)
            if child.expr.name == 'datum':
                data.append(child.text)
        self.file['data'].append(data)

    def generic_visit(self, node, visited_children):
        pass
    #return visited_children or node


In [416]:
fv = FileVisitor()

In [417]:
fv.visit(tree)


newline


In [421]:
fv.file

AttributeError: 'FileVisitor' object has no attribute 'file'

## `return` pattern

Instead of manually sticking everything into a dict, in principle we should be able to write functions that return what we want, gathering everything at the top level.

In [425]:
from parsimonious import NodeVisitor
import pandas as pd


class FileVisitor(NodeVisitor):
    
    def visit_file(self, node, visited_children):
        output = {}
        for child in visited_children:
            output.update(child[0])
        return output

    def visit_file_data(self, node, visited_children):
        well, _, country, _, date, _, company, _ = node.children
        return well

    def generic_visit(self, node, visited_children):
        return visited_children or node


In [426]:
fv = FileVisitor()

In [427]:
fv.visit(tree)

VisitationError: TypeError: 'NoneType' object is not subscriptable

Parse tree:
<Node called "file" matching "10     2                                                                                                                       
    9999/9-9                                Norway                                            9Sep99
    Weatherford-Labs
15    10   10
          1507      1602  2031   0Weatherford-Labs    Nitrogen Permeability, Hor.
          1512      1602  2031   0Weatherford-Labs    Klinkenberg corrected gas perm, Hor.
          1510      1602  2031   0Weatherford-Labs    Nitrogen Permeability, Vert.
          1515      1602  2031   0Weatherford-Labs    Klinkenberg corrected gas perm, Vert.
          1402      1211  3084   0Weatherford-Labs    Porosity, Horizontal PLUG
          1403      1211  3084   0Weatherford-Labs    Porosity, Vertical PLUG
          1401      1212  3084   0Weatherford-Labs    Porosity, Summation
          1302      1103  3085   0Weatherford-